# Proyecto: Optimización de Talento — **v6 (super comentado)**


# Optimización de Talento — 

- **Imputamos nulos**: numéricas → **mediana**; categóricas → **"desconocido"**.
- Mantenemos:
  - Texto en **minúsculas**, espacios → `_`, y `__` → `_`.
  - Limpieza de números con símbolos/formatos: **coma → punto** para decimal.
  - Columnas no negativas donde tenga sentido.
  - Columna **`sameasmonthlyincome`** con el mismo tratamiento que `monthlyincome`.

**Índice:**
1) Importaciones  
2) Preparar carpetas  
3) Leer CSV  
4) EDA básico  
5) Copia para limpieza  
6) Normalización de TEXTO  
7) Función de limpieza de NÚMEROS con símbolos  
8) Aplicar limpieza a NÚMEROS (incluye `sameasmonthlyincome`)  
9) Reglas adicionales (no negativos)  
10) Correcciones específicas (`gender`, `maritalstatus`)  
11) **Imputación de nulos** (mediana / "desconocido")  
12) Duplicados y guardado del CSV limpio  
13) Visualizaciones

In [234]:
# =========================
# 1) IMPORTACIONES
# =========================
# Importamos las librerías que vamos a usar a lo largo del cuaderno.
# - os: para trabajar con rutas y carpetas.
# - re: para expresiones regulares (nos ayudan a buscar patrones en texto).
# - sqlite3: para crear/usar una base de datos SQLite (.db) local.
# - numpy y pandas: para manipular datos.
# - matplotlib.pyplot: para hacer gráficos sencillos.

import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Ajustes de impresión para ver más columnas en pantalla (opcional).
pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 160)

print("Importaciones listas.")


Importaciones listas.


In [235]:
# =========================
# 2) PREPARAR CARPETAS
# =========================

# Imprimo la carpeta donde está corriendo el cuaderno para orientarnos.
print("Carpeta de trabajo:", os.getcwd())

# Defino la carpeta raíz donde guardaremos los datos del proyecto.
carpeta_data = "data"

# Dentro de 'data', creo una subcarpeta para los datos en bruto (sin tocar).
carpeta_raw = os.path.join(carpeta_data, "raw")

# Otra subcarpeta para los datos procesados/limpios (los resultados de la limpieza).
carpeta_processed = os.path.join(carpeta_data, "processed")

# Carpeta para guardar salidas varias (informes, textos, etc.).
carpeta_outputs = "outputs"

# Dentro de 'outputs', una subcarpeta específica para imágenes/gráficos.
carpeta_plots = os.path.join(carpeta_outputs, "plots")


# Recorro la lista de todas las carpetas que quiero que existan
# y las creo si no existen. 'exist_ok=True' evita error si ya estaban creadas.
for p in [carpeta_raw, carpeta_processed, carpeta_outputs, carpeta_plots]:
    os.makedirs(p, exist_ok=True)

# Mensaje final para confirmar las rutas que han quedado listas.
print("Carpetas listas:", [carpeta_raw, carpeta_processed, carpeta_outputs, carpeta_plots])


Carpeta de trabajo: /Users/clarabueno/Desktop/-project-da-promo-54-modulo-3-team-2
Carpetas listas: ['data/raw', 'data/processed', 'outputs', 'outputs/plots']


# Notas: LEER ANTES DE EJECUTAR LA LECTURA DE csv
- **Guardad el fichero como hr_raw_data.csv y ponedlo junto al notebook o en data/raw/. Si no, usad la celda del botón de pánico pegando vuestra ruta completa.**

In [236]:
# =========================
# 3) LEER CSV (versión simple y portable)
# =========================
# Reglas para el equipo:
# - Guardad el fichero como: hr_raw_data.csv
# - Ponedlo o bien junto al notebook, o bien en data/raw/

import os
import pandas as pd
from pathlib import Path

# 1) Rutas estándar del proyecto
ruta_local = Path("hr_raw_data.csv")                  # mismo sitio que el notebook
ruta_raw   = Path("data") / "raw" / "hr_raw_data.csv" # carpeta del proyecto

# 2) Elegimos qué ruta usar (primero local, si no, data/raw)
ruta_csv = ruta_local if ruta_local.is_file() else ruta_raw

# 3) Si no existe en ninguna, mensaje claro y listado de pistas
if not ruta_csv.is_file():
    print("⚠️ No encuentro 'hr_raw_data.csv'.")
    print("   Colócalo junto al cuaderno O en 'data/raw/hr_raw_data.csv'.")
    print("\nCarpeta de trabajo actual:", Path.cwd())
    print("\nCSVs que veo aquí:")
    for p in Path.cwd().glob("*.csv"):
        print(" -", p.name)
    raise FileNotFoundError("Falta 'hr_raw_data.csv' en las rutas esperadas.")

# 4) Leemos el CSV y copiamos una copia canónica en data/raw (para estandarizar)
df_raw = pd.read_csv(ruta_csv)

# nos aseguramos de que data/raw existe y guardamos la copia ordenada
ruta_raw.parent.mkdir(parents=True, exist_ok=True)
df_raw.to_csv(ruta_raw, index=False)

print("✅ Archivo leído desde:", ruta_csv)
print("📂 Copia estándar guardada en:", ruta_raw)
print("Tamaño DF bruto:", df_raw.shape)
df_raw.head(10)


✅ Archivo leído desde: hr_raw_data.csv
📂 Copia estándar guardada en: data/raw/hr_raw_data.csv
Tamaño DF bruto: (1678, 42)


,Unnamed: 0,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeecount,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,over18,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager,sameasmonthlyincome,datebirth,salary,roledepartament,numberchildren,remotework
0,0,51,No,NaN,2015.722222,NaN,6,3,NaN,1,1,1,0,NaN,3,5,resEArch DIREcToR,3,NaN,"16280,83$","42330,17$",7,Y,No,13,"3,0",3,Full Time,0,NaN,5,"3,0",20,NaN,15,15,"16280,83$",1972,"195370,00$",NaN,NaN,Yes
1,1,52,No,NaN,2063.388889,NaN,1,4,Life Sciences,1,2,3,0,NaN,2,5,ManAGeR,3,NaN,NaN,"43331,17$",0,NaN,NaN,14,"3,0",1,NaN,1,"34,0",5,"3,0",33,NaN,11,9,NaN,1971,"199990,00$",NaN,NaN,1
2,2,42,No,travel_rarely,1984.253968,Research & Development,4,2,Technical Degree,1,3,3,0,NaN,3,5,ManaGER,4,Married,NaN,"41669,33$",1,NaN,No,11,"3,0",4,NaN,0,"22,0",3,NaN,22,NaN,11,15,NaN,1981,"192320,00$",ManaGER - Research & Development,NaN,1
3,3,47,No,travel_rarely,1771.404762,NaN,2,4,Medical,1,4,1,1,NaN,3,4,ReseArCH DIrECtOr,3,Married,"14307,50$","37199,50$",3,Y,NaN,19,"3,0",2,Full Time,2,NaN,2,NaN,20,NaN,5,6,"14307,50$",1976,"171690,00$",NaN,NaN,False
4,4,46,No,NaN,1582.771346,NaN,3,3,Technical Degree,1,5,1,1,NaN,4,4,sAleS EXECUtIve,1,Divorced,"12783,92$","33238,20$",2,Y,No,12,"3,0",4,NaN,1,NaN,5,"3,0",19,NaN,2,8,"12783,92$",1977,NaN,NaN,NaN,0
5,5,48,No,NaN,1771.920635,Research & Development,22,3,Medical,1,6,4,1,NaN,3,4,MANAger,4,NaN,"14311,67$","37210,33$",3,NaN,No,11,"3,0",2,NaN,1,NaN,3,"3,0",22,NaN,4,7,"14311,67$",1975,NaN,MANAger - Research & Development,NaN,Yes
6,6,59,No,NaN,1032.487286,NaN,25,3,Life Sciences,1,7,1,1,NaN,3,3,Sales ExeCutIVe,1,NaN,"8339,32$","21682,23$",7,Y,NaN,11,"3,0",4,Part Time,0,"28,0",3,"2,0",21,NaN,7,9,"8339,32$",1964,"100071,84$",NaN,NaN,True
7,7,42,No,travel_rarely,556.256661,NaN,1,1,NaN,1,8,2,0,69.532083,3,2,Sales eXEcUTiVe,3,Married,NaN,"11681,39$",1,NaN,No,25,"4,0",3,Part Time,0,"20,0",3,"3,0",20,NaN,11,6,NaN,1981,"53914,11$",NaN,NaN,0
8,8,41,No,NaN,1712.182540,NaN,2,5,NaN,1,9,2,1,NaN,3,4,mANAGEr,1,Married,"13829,17$","35955,83$",7,NaN,No,16,"3,0",2,Full Time,1,"22,0",2,"3,0",18,NaN,11,8,"13829,17$",1982,"165950,00$",NaN,NaN,True
9,9,41,No,travel_frequently,1973.984127,NaN,9,3,NaN,1,10,1,0,NaN,3,5,reSEaRCH DIrectoR,3,NaN,"15943,72$","41453,67$",2,NaN,No,17,"3,0",2,NaN,1,"21,0",2,"4,0",18,NaN,0,11,"15943,72$",1982,NaN,NaN,NaN,0


In [237]:
# # =========================
# # 3) LEER CSV (ruta absoluta fácil)
# # =========================
# import pandas as pd
# from pathlib import Path
# import os

# # 👇 Pega aquí tu ruta completa entre r""
# ruta_csv_abs = r"ruta absoluta del file"

# ruta_csv = Path(ruta_csv_abs)
# if not ruta_csv.is_file():
#     raise FileNotFoundError(f"No existe el archivo en: {ruta_csv}")

# df_raw = pd.read_csv(ruta_csv)

# # Guardamos copia canónica en data/raw para el proyecto
# carpeta_raw = "data/raw"
# os.makedirs(carpeta_raw, exist_ok=True)
# destino = Path(carpeta_raw) / "hr_raw_data.csv"
# df_raw.to_csv(destino, index=False)

# print("✅ Archivo leído desde:", ruta_csv)
# print("📂 Copia estándar guardada en:", destino)
# print("Tamaño DF bruto:", df_raw.shape)
# df_raw.head(10)


In [238]:
# =========================
# 4) EDA BÁSICO
# =========================

# 1) Mostramos los tipos de datos de cada columna
# Esto nos sirve para ver si una columna que debería ser numérica está como texto, o al revés.
print("Tipos de columnas:")
print(df_raw.dtypes)

# 2) Mostramos el número de valores nulos por cada columna
# Esto nos ayuda a detectar qué columnas necesitan imputación o limpieza.
print("\nNulos por columna:")
print(df_raw.isna().sum())

# 3) Contamos cuántas filas duplicadas exactas hay en el DataFrame
# Esto es importante para evitar contar empleados más de una vez.
print("\nDuplicados exactos:", df_raw.duplicated().sum())

# 4) Guardamos esta misma información en un archivo de texto dentro de 'outputs'.
# Así tenemos un registro que podemos revisar sin necesidad de ejecutar el notebook.
with open(os.path.join(carpeta_outputs, "eda_basico_v6.txt"), "w", encoding="utf-8") as f:
    # Guardamos los tipos de columnas
    f.write("TIPOS:\n")
    f.write(str(df_raw.dtypes))
    f.write("\n\n")
    # Guardamos los nulos por columna
    f.write("NULOS:\n")
    f.write(str(df_raw.isna().sum()))
    f.write("\n")
    # Guardamos el número de duplicados
    f.write("DUPLICADOS:\n")
    f.write(str(df_raw.duplicated().sum()))

print("EDA básico guardado en 'outputs/eda_basico_v6.txt'")


Tipos de columnas:
Unnamed: 0                    int64
age                          object
attrition                    object
businesstravel               object
dailyrate                   float64
department                   object
distancefromhome              int64
education                     int64
educationfield               object
employeecount                 int64
employeenumber                int64
environmentsatisfaction       int64
gender                        int64
hourlyrate                  float64
jobinvolvement                int64
joblevel                      int64
jobrole                      object
jobsatisfaction               int64
maritalstatus                object
monthlyincome                object
monthlyrate                  object
numcompaniesworked            int64
over18                       object
overtime                     object
percentsalaryhike             int64
performancerating            object
relationshipsatisfaction      int64
standardh

In [239]:
# =========================
# 4.1) EDA INFO
# =========================

# La función .info() nos da un resumen muy útil del DataFrame:
# - El número total de filas y columnas
# - El nombre de cada columna
# - Cuántos valores NO nulos tiene cada columna
# - El tipo de dato de cada columna (int, float, object, etc.)
# - Y al final, el consumo aproximado de memoria

df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1678 entries, 0 to 1677
Data columns (total 42 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                1678 non-null   int64  
 1   age                       1678 non-null   object 
 2   attrition                 1678 non-null   object 
 3   businesstravel            877 non-null    object 
 4   dailyrate                 1678 non-null   float64
 5   department                312 non-null    object 
 6   distancefromhome          1678 non-null   int64  
 7   education                 1678 non-null   int64  
 8   educationfield            904 non-null    object 
 9   employeecount             1678 non-null   int64  
 10  employeenumber            1678 non-null   int64  
 11  environmentsatisfaction   1678 non-null   int64  
 12  gender                    1678 non-null   int64  
 13  hourlyrate                411 non-null    float64
 14  jobinvol

In [240]:
# =========================
# 5) COPIA PARA LIMPIEZA
# =========================

# 1) Creamos una copia del DataFrame original 'df_raw'.
# Esto lo hacemos para no modificar los datos originales mientras limpiamos.
# Así, si nos equivocamos o necesitamos comparar, siempre tenemos el dataset original intacto.
df = df_raw.copy()

# 2) Mostramos el tamaño (número de filas y columnas) del DataFrame de trabajo.
# df.shape devuelve una tupla (n_filas, n_columnas).
print("Tamaño inicio limpieza:", df.shape)

# 3) Mostramos las primeras 5 filas del DataFrame.
# Esto nos sirve para echar un vistazo rápido al estado inicial antes de aplicar transformaciones.
df.head(5)


Tamaño inicio limpieza: (1678, 42)


,Unnamed: 0,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeecount,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,over18,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager,sameasmonthlyincome,datebirth,salary,roledepartament,numberchildren,remotework
0,0,51,No,NaN,2015.722222,NaN,6,3,NaN,1,1,1,0,NaN,3,5,resEArch DIREcToR,3,NaN,"16280,83$","42330,17$",7,Y,No,13,"3,0",3,Full Time,0,NaN,5,"3,0",20,NaN,15,15,"16280,83$",1972,"195370,00$",NaN,NaN,Yes
1,1,52,No,NaN,2063.388889,NaN,1,4,Life Sciences,1,2,3,0,NaN,2,5,ManAGeR,3,NaN,NaN,"43331,17$",0,NaN,NaN,14,"3,0",1,NaN,1,"34,0",5,"3,0",33,NaN,11,9,NaN,1971,"199990,00$",NaN,NaN,1
2,2,42,No,travel_rarely,1984.253968,Research & Development,4,2,Technical Degree,1,3,3,0,NaN,3,5,ManaGER,4,Married,NaN,"41669,33$",1,NaN,No,11,"3,0",4,NaN,0,"22,0",3,NaN,22,NaN,11,15,NaN,1981,"192320,00$",ManaGER - Research & Development,NaN,1
3,3,47,No,travel_rarely,1771.404762,NaN,2,4,Medical,1,4,1,1,NaN,3,4,ReseArCH DIrECtOr,3,Married,"14307,50$","37199,50$",3,Y,NaN,19,"3,0",2,Full Time,2,NaN,2,NaN,20,NaN,5,6,"14307,50$",1976,"171690,00$",NaN,NaN,False
4,4,46,No,NaN,1582.771346,NaN,3,3,Technical Degree,1,5,1,1,NaN,4,4,sAleS EXECUtIve,1,Divorced,"12783,92$","33238,20$",2,Y,No,12,"3,0",4,NaN,1,NaN,5,"3,0",19,NaN,2,8,"12783,92$",1977,NaN,NaN,NaN,0


In [241]:
# =========================
# 6) LIMPIEZA: ELIMINAR COLUMNAS
# =========================

# Lista de columnas que queremos eliminar
columnas_a_eliminar = [
    "employeecount",
    "yearsincurrentrole",
    "sameasmonthlyincome",
    "roledepartament",
    "numberchildren",
    "over18"
]

# Comprobamos cuáles de esas columnas están realmente en el DataFrame
columnas_presentes = [col for col in columnas_a_eliminar if col in df.columns]

# Si hay columnas presentes, las eliminamos
if columnas_presentes:
    df.drop(columns=columnas_presentes, inplace=True)
    print(f"Columnas eliminadas: {columnas_presentes}")
else:
    print("Ninguna de las columnas especificadas está presente en el DataFrame.")

Columnas eliminadas: ['employeecount', 'yearsincurrentrole', 'sameasmonthlyincome', 'roledepartament', 'numberchildren', 'over18']


In [242]:
mask = df.duplicated(subset="employeenumber")
df[mask]

,Unnamed: 0,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearssincelastpromotion,yearswithcurrmanager,datebirth,salary,remotework
1614,1614,35,No,NaN,1032.487286,NaN,18,5,Life Sciences,178,2,0,NaN,3,3,sAlES eXeCuTIVe,1,Married,"8339,32$","21682,23$",1,NaN,22,"4,0",4,NaN,1,"9,0",3,"2,0",9,1,8,1988,NaN,True
1615,1615,59,No,travel_rarely,290.035510,Human Resources,6,2,Medical,853,2,0,NaN,3,1,humAN resoURCEs,3,Married,"2342,59$","6090,75$",8,NaN,17,"3,0",4,Part Time,0,NaN,2,"2,0",2,2,2,1964,"28111,13$",False
1616,1616,30,No,travel_rarely,1032.487286,NaN,5,3,NaN,112,2,1,129.060911,3,3,SalES ExeCuTIVe,4,NaN,"8339,32$","21682,23$",2,No,12,"3,0",3,Part Time,1,NaN,2,"3,0",10,7,4,1993,"100071,84$",True
1617,1617,34,Yes,travel_rarely,556.256661,NaN,24,4,NaN,159,1,1,NaN,2,2,sAles EXecutiVE,2,Single,NaN,"11681,39$",0,Yes,23,"4,0",3,Part Time,0,"16,0",2,"4,0",15,10,10,1989,"53914,11$",0
1618,1618,47,No,NaN,290.035510,NaN,2,4,Life Sciences,969,4,1,NaN,2,1,SALeS RePreSenTATiVe,4,Single,"2342,59$","6090,75$",1,Yes,18,"3,0",1,NaN,0,"3,0",3,"2,0",3,1,2,1976,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1673,1673,43,No,NaN,488.944444,NaN,-26,3,Medical,824,2,1,NaN,4,1,rESEaRcH SciEnTiST,3,Single,"3949,17$","10267,83$",4,NaN,12,"3,0",4,NaN,0,NaN,2,"3,0",3,1,2,1980,NaN,Yes
1674,1674,47,No,NaN,1973.984127,NaN,26,4,NaN,1087,4,1,NaN,3,5,mANager,3,Married,"15943,72$","41453,67$",3,No,11,"3,0",3,Full Time,1,"27,0",2,"3,0",5,1,0,1976,"191324,62$",False
1675,1675,29,No,travel_rarely,290.035510,NaN,15,3,NaN,528,3,0,NaN,3,1,reSearch sCienTiSt,4,NaN,NaN,"6090,75$",1,No,19,"3,0",1,Part Time,0,"6,0",1,"3,0",6,1,5,1994,"28111,13$",False
1676,1676,47,No,travel_rarely,1032.487286,NaN,4,3,Life Sciences,76,3,1,NaN,2,3,maNufACTURING DIREctOr,2,Divorced,"8339,32$","21682,23$",8,Yes,12,NaN,3,Part Time,1,NaN,4,"3,0",22,14,10,1976,"100071,84$",Yes


In [243]:
# =========================
# 10) FUNCIÓN: limpiar números con símbolos
# =========================
# Esta función limpia valores numéricos escritos como texto:
# - Quita símbolos ($, €, %, espacios).
# - Arregla el uso de comas y puntos (formatos europeo/USA).
# - Deja solo dígitos y el punto decimal.

def limpiar_numero_texto(x):
    # Si es NaN, lo devolvemos tal cual (se imputará luego).
    if pd.isna(x):
        return x

    # Convertimos a string para poder reemplazar. para poder manipularlo con replace()
    s = str(x)

    # Quito espacios, símbolos de moneda ($, €) y porcentajes (%)
    s = s.replace(" ", "").replace("$", "").replace("€", "").replace("%", "")

    # Gestionamos comas y puntos; Si hay coma y punto a la vez, tengo que decidir cuál es el decimal
    if "," in s and "." in s:
        # Si termina con ,dd asumimos formato europeo, ej: 1.234,56 -> 1234.56
        if re.search(r",\d{1,2}$", s): # Caso europeo: si termina en coma + 1 o 2 dígitos (ej. "1.234,56")
            s = s.replace(".", "").replace(",", ".") # Quito los puntos de miles y convierto la coma en punto decimal
        else:
            # Si no parece europeo, quitamos comas como separador de miles
            # Caso americano: la coma es separador de miles, la borro
            # Ejemplo: "1,234.56" -> "1234.56"
            s = s.replace(",", "")
    else:
        # Si solo hay coma, la tratamos como decimal, ej: 123,45 -> 123.45
        if "," in s and "." not in s:
            # Asumo que la coma es decimal -> la cambio a punto
            # También elimino puntos sueltos por si acaso
            s = s.replace(".", "").replace(",", ".")

    # Limpieza final con regex:
    # Borro cualquier cosa que no sea número o punto decimal
    # # Nos quedamos solo con dígitos y el punto
    s = re.sub(r"[^0-9.]", "", s)

    # Devuelvo el string limpio (más tarde lo convertiremos a float)
    return s

print("Función 'limpiar_numero_texto' lista.")


Función 'limpiar_numero_texto' lista.


In [244]:
# =========================
# 7) LIMPIEZA NÚMEROS (aplicar función y convertir a numérico)
# =========================

# Lista de columnas que esperamos numéricas
# Primero hago una lista con los nombres de las columnas que deberían ser numéricas en nuestro dataset.
# Estas columnas, en bruto, pueden venir con símbolos de dinero, comas en lugar de puntos, etc.
# También incluyo 'sameasmonthlyincome', que debe tratarse igual que 'monthlyincome'.
columnas_numericas_esperadas = [
    "dailyrate","hourlyrate","monthlyincome","sameasmonthlyincome","monthlyrate",
    "percentsalaryhike","stockoptionlevel","distancefromhome",
    "age","education","joblevel","totalworkingyears",
    "yearsatcompany","yearsincurrentrole","yearssincelastpromotion",
    "yearswithcurrmanager","numberchildren","salary","trainingtimeslastyear",
    "environmentsatisfaction","jobinvolvement","jobsatisfaction",
    "relationshipsatisfaction","performancerating","worklifebalance"
]

# Creamos un mapa lower -> nombre real de la columna en el DF
# Como en el dataset los nombres de columnas pueden variar en mayúsculas/minúsculas,
# creo un diccionario que relaciona el nombre en minúsculas con el nombre real en el DataFrame.
# Ejemplo: si la columna se llama "DailyRate", este mapa me devuelve "DailyRate" cuando busco "dailyrate".
lower_map = {c.lower(): c for c in df.columns}

# Recorremos la lista y, si la columna existe, la limpiamos (recorro cada nombre de la lista que hice arriba.)
for cname in columnas_numericas_esperadas:
# Si esa columna existe realmente en el DataFrame (aunque esté en mayúsculas),
# entonces la limpiamos
    if cname in lower_map: 
        # Recupero el nombre real de la columna
        col = lower_map[cname]

        # 1) Convierto la columna a texto (string) para poder aplicar la función de limpieza sin errores.
        df[col] = df[col].astype("string")
        
        # 2) Aplico nuestra función 'limpiar_numero_texto' que elimina símbolos ($, €, %) y arregla comas/puntos.
        df[col] = df[col].apply(limpiar_numero_texto)
        # 3) Convierto el resultado a número real (float).
        #    Si alguna celda no se puede convertir, se pone como NaN automáticamente.
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Mensaje final para confirmar que la limpieza se ha aplicado a todas las columnas numéricas.
print("Limpieza numérica aplicada (decimal='.') y conversión a numérico.")

Limpieza numérica aplicada (decimal='.') y conversión a numérico.


In [245]:
# =========================
# 8) CORRECCIONES ESPECÍFICAS
# =========================

# --- Corrección 1: columna 'gender'
# En algunos datasets, 'gender' aparece como 0/1 en vez de 'male'/'female'.
# Por claridad, convertimos 1 -> "male" y 0 -> "female".
if "gender" in lower_map:                       # solo si existe la columna gender
    col = lower_map["gender"]                   # recupero el nombre real de la columna
    if pd.api.types.is_numeric_dtype(df[col]):  # si es numérica (0/1)
        df[col] = df[col].map({1:"male", 0:"female"})
    else:
        # Si viene como texto "0" o "1" (string), también lo convertimos
        df[col] = df[col].replace({"1":"male","0":"female",1:"male",0:"female"})

# --- Corrección 2: columna 'maritalstatus'
# A veces hay un error tipográfico: en vez de "married" aparece "marreid".
# Lo corregimos para que todos los valores estén uniformes.
if "maritalstatus" in lower_map:                        # solo si existe la columna maritalstatus
    col = lower_map["maritalstatus"]
    # Convertimos a string y reemplazamos "marreid" por "married"
    df[col] = df[col].astype("string").str.replace("marreid","married",regex=False)

# --- Corrección 3: columna 'remotework'
# La columna 'remotework' contenía valores redundantes para expresar lo mismo 
# (true/false, 0/1, yes/no). 
# Se unifican y normalizan todos ellos en un formato único: 'yes' / 'no'.
if "remotework" in lower_map:                        # solo si existe la columna remotework
    col = lower_map["remotework"]
    # Convertimos a string y reemplazamos
    df[col] = df[col].replace({"true":"yes", "True":"yes", "false":"no", "0":"no", "1":"yes", "False":"no", "Yes":"yes"})

# Mensaje de confirmación para saber que ya hicimos estas correcciones puntuales.
print("Correcciones (gender, maritalstatus, remotework) aplicadas.")

Correcciones (gender, maritalstatus, remotework) aplicadas.


In [246]:
# =========================
# 9) NORMALIZACIÓN DE TEXTO 
# =========================
# Reglas aplicadas a TODAS las columnas categóricas:
# - Convertir todo a string (así evitamos mezclas).
# - strip(): quitar espacios en los bordes.
# - lower(): pasar todo a minúsculas.
# - Reemplazar espacios por '_' (guión bajo).
# - Colapsar '__' a '_' (por si había dobles espacios).
# - Quitar '_' al principio y al final.
# - Reemplazar vacíos, 'nan', 'none' por NaN real (pd.NA).

for col in df.columns:
    if not pd.api.types.is_numeric_dtype(df[col]):
        # 1) Convertimos TODO a string para que funcione bien el .str
        df[col] = df[col].astype("string")
        
        # 2) Normalizamos
        df[col] = df[col].str.strip()
        df[col] = df[col].str.lower()
        df[col] = df[col].str.replace(" ", "_", regex=False)
        df[col] = df[col].str.replace(r"_+", "_", regex=True)
        df[col] = df[col].str.replace("-", "_", regex=False)
        df[col] = df[col].str.strip("_")
        
        # 3) Sustitución de vacíos por NaN real
        df[col] = df[col].replace({"": pd.NA, "nan": pd.NA, "none": pd.NA})

print("Normalización de texto aplicada a TODAS las columnas no numéricas.")

Normalización de texto aplicada a TODAS las columnas no numéricas.


### 🔹 ¿Qué es una escala **Likert**?

👉 

* Es una **escala de encuesta muy común** en psicología y en recursos humanos.
* Normalmente se usa para medir **opiniones o percepciones subjetivas** en valores discretos (números enteros).
* Ejemplo típico:

  * 1 = Muy insatisfecho
  * 2 = Insatisfecho
  * 3 = Satisfecho
  * 4 = Muy satisfecho

En nuestro dataset, hay varias columnas que funcionan exactamente así:

* **EnvironmentSatisfaction** (satisfacción con el ambiente laboral)
* **JobSatisfaction** (satisfacción con el trabajo)
* **RelationshipSatisfaction** (satisfacción con relaciones en el trabajo)
* **JobInvolvement** (compromiso con el trabajo)
* **WorkLifeBalance** (equilibrio vida personal/laboral)

Todas deberían contener **sólo 1, 2, 3 o 4**.
Si vemos un 37, 42 o algo extraño → es un **error en los datos** → lo convertimos en `NaN` para luego imputarlo.

In [247]:
# =========================
# 11) REGLAS: LIKERT (1..4) + NO NEGATIVOS
# =========================
# CONTEXTO:
# Hay dos tipos de “normas de sentido común” que debemos aplicar al dataset:
#
# A) ESCALAS LIKERT (1..4)
#    - Varias columnas provienen de encuestas internas (satisfacción, implicación, conciliación).
#    - Esas encuestas se responden en una ESCALA LIKERT de 4 puntos:
#         1 = muy insatisfecho / muy bajo
#         2 = insatisfecho / bajo
#         3 = satisfecho / alto
#         4 = muy satisfecho / muy alto
#    - Por lo tanto, en esas columnas SOLO deberían aparecer los valores {1, 2, 3, 4}.
#    - Si encontramos otros números (ej. 37, 42) o textos raros, es un error de datos.
#    - Solución: marcamos esos valores como NaN para imputarlos más tarde (con la mediana).
#
# B) COLUMNAS NO NEGATIVAS
#    - Hay métricas que por lógica no pueden ser negativas (distancias, sueldos, tarifas).
#    - Si por error hay un valor negativo, lo convertimos a su valor absoluto (abs()).

# --- A) Listado de columnas tipo Likert (1..4)
likert_cols = [
    "environmentsatisfaction",   # satisfacción con el ambiente laboral
    "jobsatisfaction",           # satisfacción general con el trabajo
    "relationshipsatisfaction",  # satisfacción con relaciones en el trabajo
    "jobinvolvement",            # implicación/compromiso con el trabajo
    "worklifebalance"            # equilibrio vida-trabajo
]

for cname in likert_cols:
    # Comprobamos que la columna exista en el DataFrame (usamos lower_map por si el nombre viene con mayúsculas)
    if cname in lower_map:
        col = lower_map[cname]  # nombre real de la columna tal como está en df: Recupero el nombre real de la columna (con mayúsculas/minúsculas como esté en el CSV).

        # Solo aplicamos la validación si la columna es numérica de verdad.
        if pd.api.types.is_numeric_dtype(df[col]):
            # Aplico el valor absoluto a toda la columna.
            # Creamos una máscara booleana que es True donde el valor NO está en {1,2,3,4}
            # Es decir, detectamos valores incorrectos para una escala Likert de 4 puntos.
            fuera_de_rango = ~df[col].isin([1, 2, 3, 4])

            # A esos valores incorrectos les ponemos NaN (pd.NA) para arreglarlos después en la imputación.
            df.loc[fuera_de_rango, col] = pd.NA

# --- B) Columnas que no deberían ser negativas por sentido común
no_negativas = [
    "distancefromhome",      # distancia desde casa al trabajo
    "dailyrate",             # tarifa diaria
    "hourlyrate",            # tarifa por hora
    "monthlyincome",         # ingreso mensual
    "sameasmonthlyincome",   # (si existe) duplicado de monthlyincome
    "monthlyrate",           # tarifa mensual
    "salary"                 # salario anual/mensual (según dataset)
]

for cname in no_negativas:
    if cname in lower_map:
        col = lower_map[cname]

        # Solo aplicamos si pandas la ve como numérica
        if pd.api.types.is_numeric_dtype(df[col]):
            # Valor absoluto: si hubiera -500 por error, lo convertimos a 500
            df[col] = df[col].abs()

# Mensaje final de confirmación para saber que la regla ya se aplicó.
print("Reglas aplicadas: Likert limitado a {1,2,3,4} y métricas no negativas forzadas a valores ≥ 0.")


Reglas aplicadas: Likert limitado a {1,2,3,4} y métricas no negativas forzadas a valores ≥ 0.


In [248]:
# =========================
# 12) IMPUTACIÓN DE NULOS
# =========================
# - Numéricas: usamos la MEDIANA de la columna (es robusta frente a outliers).
#   Si toda la columna es NaN, usamos 0.0 (especialmente si son métricas no-negativas).
# - Categóricas (texto): rellenamos con la palabra 'desconocido'.

# 1) Detectamos numéricas y categóricas por dtype
num_cols = df.select_dtypes(include=[np.number]).columns   # columnas numéricas
cat_cols = df.select_dtypes(exclude=[np.number]).columns   # columnas no numéricas (texto)

# 2) Lista de columnas que deben ser NO NEGATIVAS (para elegir fallback sensato si todo es NaN)
no_neg_set = {
    "distancefromhome","dailyrate","hourlyrate","monthlyincome","sameasmonthlyincome",
    "monthlyrate","salary","percentsalaryhike","stockoptionlevel","numberchildren",
    "trainingtimeslastyear","yearsatcompany","yearsincurrentrole","yearssincelastpromotion",
    "yearswithcurrmanager","totalworkingyears","age","education","joblevel",
}

# Imputación numéricas
for c in num_cols:
    if df[c].isna().all():
        # Si toda la columna es NaN, usamos 0.0 (simple y válido para estas métricas).
        fill_value = 0.0 if c.lower() in no_neg_set else 0.0
    else:
        # Si hay datos, usamos la mediana (ignora NaN por defecto).
        fill_value = df[c].median(skipna=True)
    df[c] = df[c].fillna(fill_value)

# Imputación categóricas
for c in cat_cols:
    df[c] = df[c].fillna("desconocido")

print("Imputación completada (numéricas=mediana/fallback, categóricas='desconocido').")


Imputación completada (numéricas=mediana/fallback, categóricas='desconocido').


In [249]:
# =========================
# 13) CHEQUEO RÁPIDO TRAS IMPUTACIÓN (opcional)
# =========================
# OBJETIVO DE ESTA CELDA:
# - Verificar que la imputación de nulos ha funcionado.
# - Un vistazo rápido a los datos resultantes:
#   * ¿Quedan columnas con nulos?
#   * Ver un pequeño resumen de una columna numérica (describe)
#   * Ver las categorías más frecuentes de una columna de texto

# -------------------------------------------------------------------
# 1) ¿Quedan nulos en alguna columna?
# -------------------------------------------------------------------

# 'df.isna().sum()' devuelve cuántos nulos hay en cada columna.
nulos_restantes = df.isna().sum()

# Contamos cuántas columnas tienen al menos 1 nulo (True/False -> sum de booleanos).
n_cols_con_nulos = int((nulos_restantes > 0).sum())

# Mostramos el número total de columnas con nulos todavía.
print(f"Columnas con nulos restantes: {n_cols_con_nulos}")

# Si queda alguna con nulos, enseñamos las 10 peores (más nulos) para revisar.
if n_cols_con_nulos > 0:
    print("\nColumnas con más nulos tras la imputación (top 10):")
    print(
        nulos_restantes[nulos_restantes > 0]         # nos quedamos solo con las que tienen nulos
        .sort_values(ascending=False)                # ordenamos de mayor a menor
        .head(10)                                    # mostramos las 10 primeras
    )

# -------------------------------------------------------------------
# 2) Hacemos un mini-resumen de ejemplo:
#    - elegimos UNA columna numérica y UNA categórica
#    - esto es solo para tener una referencia rápida en la consola
# -------------------------------------------------------------------

# a) Buscamos la primera columna que pandas considere numérica.
#    'next(..., None)' devuelve el primer elemento que cumpla la condición; si no hay, devuelve None.
ej_num = next((c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])), None)

# b) Buscamos la primera columna que NO sea numérica (o sea, categórica/texto).
ej_cat = next((c for c in df.columns if not pd.api.types.is_numeric_dtype(df[c])), None)

# Si hemos encontrado alguna numérica, mostramos su 'describe()'
# (cuenta, media, desviación estándar, mínimos, cuartiles…)
if ej_num:
    print(f"\nResumen numérico de '{ej_num}':")
    print(df[ej_num].describe())

# Si hemos encontrado alguna categórica, mostramos sus 10 valores más frecuentes.
if ej_cat:
    print(f"\nTop categorías de '{ej_cat}':")
    print(df[ej_cat].value_counts().head(10))

# NOTA:
# Este chequeo es "rápido" y orientativo. Si quieres un control de calidad más completo,
# podrías guardar un informe con más métricas (distribuciones, outliers, etc.) en 'outputs/'.

Columnas con nulos restantes: 0

Resumen numérico de 'Unnamed: 0':
count    1678.000000
mean      838.500000
std       484.541192
min         0.000000
25%       419.250000
50%       838.500000
75%      1257.750000
max      1677.000000
Name: Unnamed: 0, dtype: float64

Top categorías de 'attrition':
attrition
no     1406
yes     272
Name: count, dtype: Int64


In [250]:
# Funciones de agregación personalizadas
def resolve_remote_work(values):
    """Si hay discrepancias en Remote Work, siempre devolver 'No'."""
    return "no" if "no" in values.values else values.iloc[0]
def resolve_distance(values):
    """Si hay discrepancias en Distance From Home, tomar el mínimo."""
    return values.min()
# Diccionario de agregaciones
agg_rules = {
    "remotework": resolve_remote_work,
    "distancefromhome": resolve_distance
}
# Para las demás columnas, aplicamos 'first' por defecto
for col in df.columns:
    if col not in ["employeenumber", "remotework", "distancefromhome"]:
        agg_rules[col] = "first"
# Agrupar y aplicar reglas
df_limpio = df.groupby("employeenumber", as_index=False).agg(agg_rules)
print(":white_tick: DataFrame limpio creado:")
print(df_limpio.head())








:white_tick: DataFrame limpio creado:
   employeenumber remotework  distancefromhome  Unnamed: 0   age attrition businesstravel    dailyrate              department  education    educationfield  \
0               1        yes                 6           0  51.0        no    desconocido  2015.722222             desconocido          3       desconocido   
1               2        yes                 1           1  52.0        no    desconocido  2063.388889             desconocido          4     life_sciences   
2               3        yes                 4           2  42.0        no  travel_rarely  1984.253968  research_&_development          2  technical_degree   
3               4         no                 2           3  47.0        no  travel_rarely  1771.404762             desconocido          4           medical   
4               5         no                 3           4  46.0        no    desconocido  1582.771346             desconocido          3  technical_degree   

   envi

In [251]:
# =========================
# 14) DUPLICADOS
# =========================
# OBJETIVO:
# 1) Eliminar filas duplicadas exactas (todas las columnas iguales).
# 2) Eliminar duplicados por 'EmployeeNumber' (clave de empleado), conservando la PRIMERA fila.
#    -> Esto evita errores al cargar en BBDD, donde 'EmployeeNumber' será PRIMARY KEY.
# 3) Guardar un CSV con los registros descartados para poder revisarlos rapido.

from pathlib import Path

# --- Paso 0: localizar la columna EmployeeNumber sin importar mayúsculas ---
# Creamos un mapa lower -> nombre real de la columna
lower_map = {c.lower(): c for c in df.columns}

# Detectamos el nombre correcto tal y como está en el DF
# buscamos ()'employeenumber' o variaciones como 'employee_number') porque no me fio ya de lo que haya en la DB
emp_candidates = ["employeenumber", "employee_number", "employee number"]
emp_col = next((lower_map[c] for c in emp_candidates if c in lower_map), None)

if emp_col is None:
    # Si no existe la columna, avisa y solo haz el deduplicado exacto de filas
    print("⚠️ No existe columna de EmployeeNumber en el DF. Solo eliminamos duplicados exactos de filas.")
    antes = df.shape[0]
    df = df.drop_duplicates(keep="first")
    despues = df.shape[0]
    print(f"Duplicados EXACTOS eliminados: {antes - despues}")
else:
    # ---------------------------------------------------------------
    # 1) Eliminar duplicados exactos de fila (todas las columnas iguales)
    # ---------------------------------------------------------------
    antes = df.shape[0]
    df = df.drop_duplicates(keep="first")
    despues = df.shape[0]
    print(f"Duplicados EXACTOS eliminados: {antes - despues}")

    # ---------------------------------------------------------------
    # 2) Eliminar duplicados por EmployeeNumber (conservar la primera)
    # ---------------------------------------------------------------
    # a) Calculamos qué filas son duplicadas de EmployeeNumber (a partir de la segunda por la cabecera, logicamente)
    #    -> df.duplicated(subset=emp_col, keep='first') da True en las filas que sobran, como nos dice Laura.
    mask_duplicados_emp = df.duplicated(subset=emp_col, keep="first")

    # b) Guardamos las filas que VAMOS A DESCARTAR para revisión/ auditoría
    descartadas = df.loc[mask_duplicados_emp].copy()

    # c) Nos quedamos solo con la PRIMERA fila de cada EmployeeNumber
    df = df.drop_duplicates(subset=emp_col, keep="first")

    # d) Mensajes resumen
    n_descartadas = descartadas.shape[0]
    print(f"Duplicados por '{emp_col}' eliminados (conservando primera aparición): {n_descartadas}")

    # ---------------------------------------------------------------
    # 3) Guardar las filas descartadas para poder revisarlas en equipo
    # ---------------------------------------------------------------
    verif_dir = Path(carpeta_outputs) / "verification"
    verif_dir.mkdir(parents=True, exist_ok=True)
    ruta_desc = verif_dir / f"empleados_descartados_por_duplicado_{emp_col}.csv"
    if n_descartadas > 0:
        descartadas.to_csv(ruta_desc, index=False)
        print("📄 Registros descartados guardados en:", ruta_desc)
    else:
        print("No había duplicados por EmployeeNumber. No se generó archivo de descartes.")

# Vistazo rápido tras la limpieza de duplicados
print("Tamaño DF tras deduplicar:", df.shape)
df.head(5)

Duplicados EXACTOS eliminados: 0
Duplicados por 'employeenumber' eliminados (conservando primera aparición): 64
📄 Registros descartados guardados en: outputs/verification/empleados_descartados_por_duplicado_employeenumber.csv
Tamaño DF tras deduplicar: (1614, 36)


,Unnamed: 0,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearssincelastpromotion,yearswithcurrmanager,datebirth,salary,remotework
0,0,51.0,no,desconocido,2015.722222,desconocido,6,3,desconocido,1,1.0,female,69.532083,3.0,5,research_director,3.0,desconocido,16280.83,42330.17,7,no,13,3.0,3.0,full_time,0,10.0,5,3.0,20,15,15,1972,195370.00,yes
1,1,52.0,no,desconocido,2063.388889,desconocido,1,4,life_sciences,2,3.0,female,69.532083,2.0,5,manager,3.0,desconocido,4492.84,43331.17,0,desconocido,14,3.0,1.0,desconocido,1,34.0,5,3.0,33,11,9,1971,199990.00,yes
2,2,42.0,no,travel_rarely,1984.253968,research_&_development,4,2,technical_degree,3,3.0,female,69.532083,3.0,5,manager,4.0,married,4492.84,41669.33,1,no,11,3.0,4.0,desconocido,0,22.0,3,3.0,22,11,15,1981,192320.00,yes
3,3,47.0,no,travel_rarely,1771.404762,desconocido,2,4,medical,4,1.0,male,69.532083,3.0,4,research_director,3.0,married,14307.50,37199.50,3,desconocido,19,3.0,2.0,full_time,2,10.0,2,3.0,20,5,6,1976,171690.00,no
4,4,46.0,no,desconocido,1582.771346,desconocido,3,3,technical_degree,5,1.0,male,69.532083,4.0,4,sales_executive,1.0,divorced,12783.92,33238.20,2,no,12,3.0,4.0,desconocido,1,10.0,5,3.0,19,2,8,1977,53914.11,no


In [252]:
# =========================
# 16) GUARDADO
# =========================

# -------------------------------------------------------------------
# Guardar el CSV limpio en 'data/processed'
# -------------------------------------------------------------------

# Construimos la ruta completa del archivo de salida (dentro de la carpeta 'processed').
ruta_csv_limpio = os.path.join(carpeta_processed, "hr_clean_data_v6.csv")

# Guardamos el DataFrame limpio en formato CSV.
# - index=False: no guardamos el índice de pandas como columna extra.
df.to_csv(ruta_csv_limpio, index=False)

# Mensaje para confirmar dónde se guardó el archivo.
print("CSV limpio v6 guardado en:", ruta_csv_limpio)

# -------------------------------------------------------------------
# 3) Vistazo rápido a las primeras filas (solo para comprobar)
# -------------------------------------------------------------------

# Mostramos las primeras 10 filas para verificar visualmente que todo tiene buena pinta.
df.head(10)

CSV limpio v6 guardado en: data/processed/hr_clean_data_v6.csv


,Unnamed: 0,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearssincelastpromotion,yearswithcurrmanager,datebirth,salary,remotework
0,0,51.0,no,desconocido,2015.722222,desconocido,6,3,desconocido,1,1.0,female,69.532083,3.0,5,research_director,3.0,desconocido,16280.83,42330.17,7,no,13,3.0,3.0,full_time,0,10.0,5,3.0,20,15,15,1972,195370.00,yes
1,1,52.0,no,desconocido,2063.388889,desconocido,1,4,life_sciences,2,3.0,female,69.532083,2.0,5,manager,3.0,desconocido,4492.84,43331.17,0,desconocido,14,3.0,1.0,desconocido,1,34.0,5,3.0,33,11,9,1971,199990.00,yes
2,2,42.0,no,travel_rarely,1984.253968,research_&_development,4,2,technical_degree,3,3.0,female,69.532083,3.0,5,manager,4.0,married,4492.84,41669.33,1,no,11,3.0,4.0,desconocido,0,22.0,3,3.0,22,11,15,1981,192320.00,yes
3,3,47.0,no,travel_rarely,1771.404762,desconocido,2,4,medical,4,1.0,male,69.532083,3.0,4,research_director,3.0,married,14307.50,37199.50,3,desconocido,19,3.0,2.0,full_time,2,10.0,2,3.0,20,5,6,1976,171690.00,no
4,4,46.0,no,desconocido,1582.771346,desconocido,3,3,technical_degree,5,1.0,male,69.532083,4.0,4,sales_executive,1.0,divorced,12783.92,33238.20,2,no,12,3.0,4.0,desconocido,1,10.0,5,3.0,19,2,8,1977,53914.11,no
5,5,48.0,no,desconocido,1771.920635,research_&_development,22,3,medical,6,4.0,male,69.532083,3.0,4,manager,4.0,desconocido,14311.67,37210.33,3,no,11,3.0,2.0,desconocido,1,10.0,3,3.0,22,4,7,1975,53914.11,yes
6,6,59.0,no,desconocido,1032.487286,desconocido,25,3,life_sciences,7,1.0,male,69.532083,3.0,3,sales_executive,1.0,desconocido,8339.32,21682.23,7,desconocido,11,3.0,4.0,part_time,0,28.0,3,2.0,21,7,9,1964,100071.84,yes
7,7,42.0,no,travel_rarely,556.256661,desconocido,1,1,desconocido,8,2.0,female,69.532083,3.0,2,sales_executive,3.0,married,4492.84,11681.39,1,no,25,4.0,3.0,part_time,0,20.0,3,3.0,20,11,6,1981,53914.11,no
8,8,41.0,no,desconocido,1712.182540,desconocido,2,5,desconocido,9,2.0,male,69.532083,3.0,4,manager,1.0,married,13829.17,35955.83,7,no,16,3.0,2.0,full_time,1,22.0,2,3.0,18,11,8,1982,165950.00,yes
9,9,41.0,no,travel_frequently,1973.984127,desconocido,9,3,desconocido,10,1.0,female,69.532083,3.0,5,research_director,3.0,desconocido,15943.72,41453.67,2,no,17,3.0,2.0,desconocido,1,21.0,2,4.0,18,0,11,1982,53914.11,no


In [253]:
# =========================
# 16) VISUALIZACIONES
# =========================
# En esta sección creamos varios gráficos sencillos para entender mejor el dataset.
# Cada gráfico se guarda como imagen dentro de la carpeta 'plots' para usarlo en la demo.

# --- Visualización 1: Gráfico de barras de 'attrition' ---
# Comprobamos si existe la columna 'attrition' en el DataFrame
if "attrition" in df.columns:
    plt.figure()  # abrimos una figura nueva (lienzo en blanco)
    # Contamos los valores de 'attrition' (sí/no) y los mostramos en barras.
    # 'dropna=False' incluye los posibles NaN en el conteo (por transparencia).
    df["attrition"].value_counts(dropna=False).plot(kind="bar")
    plt.title("Attrition (conteo)")             # título del gráfico
    plt.xlabel("attrition")                     # etiqueta eje X
    plt.ylabel("n")                             # etiqueta eje Y (número de empleados)
    # Guardamos la imagen en la carpeta 'plots' para tenerla disponible en la demo
    p1 = os.path.join(carpeta_plots, "attrition_counts_v6.png")
    plt.savefig(p1, bbox_inches="tight")        # bbox_inches ajusta márgenes para que no se corte
    plt.close()                                 # cerramos la figura para liberar memoria
    print("Gráfico:", p1)                       # confirmamos la ruta del archivo generado

# --- Visualización 2: Histograma de 'monthlyincome' ---
# Comprobamos si existe la columna 'monthlyincome'
if "monthlyincome" in df.columns:
    plt.figure()  # abrimos figura nueva
    # Dibujamos un histograma con 30 bins (intervalos) para ver la distribución de ingresos
    # dropna() quita NaN; astype(float) asegura tipo numérico para el histograma
    df["monthlyincome"].dropna().astype(float).plot(kind="hist", bins=30)
    plt.title("MonthlyIncome (histograma)")     # título
    plt.xlabel("monthlyincome")                 # eje X: ingresos mensuales
    plt.ylabel("frecuencia")                    # eje Y: cuántos empleados caen en cada rango
    # Guardamos la imagen
    p2 = os.path.join(carpeta_plots, "monthlyincome_hist_v6.png")
    plt.savefig(p2, bbox_inches="tight")
    plt.close()
    print("Gráfico:", p2)

# --- Visualización 3: Boxplot de 'yearsatcompany' por 'department' ---
# Solo si tenemos ambas columnas disponibles (años en la empresa y departamento)
if "yearsatcompany" in df.columns and "department" in df.columns:
    plt.figure()                 # nueva figura
    data = []                    # lista con los arrays de valores por departamento
    labels = []                  # nombres (etiquetas) de cada departamento

    # Agrupamos por departamento y recogemos los valores de 'yearsatcompany'
    for d, sub in df.groupby("department"):
        # Convertimos a float y quitamos nulos con dropna()
        vals = sub["yearsatcompany"].dropna().astype(float).values
        if len(vals) > 0:
            data.append(vals)         # añadimos la distribución de ese departamento
            labels.append(str(d))     # guardamos el nombre del departamento

    # Si tenemos datos, dibujamos el boxplot comparativo (uno por departamento)
    if data:
        plt.boxplot(data, labels=labels, vert=True)  # vert=True para cajas verticales
        plt.xticks(rotation=45, ha="right")          # giramos etiquetas del eje X para que no se solapen
        plt.tight_layout()                           # ajustamos diseño para que quepa todo
        # Guardamos la imagen del boxplot
        p3 = os.path.join(carpeta_plots, "yearsatcompany_by_department_v6.png")
        plt.savefig(p3, bbox_inches="tight")
        plt.close()
        print("Gráfico:", p3)

# --- Visualización 4: Attrition por JobRole (porcentaje de 'yes') ---
# Muestra qué roles tienen mayor rotación relativa (porcentaje de bajas).
if "attrition" in df.columns and "jobrole" in df.columns:
    plt.figure()  # nueva figura

    # Calculamos el % de 'yes' por rol:
    # 1) Creamos una columna auxiliar 'attr' en string
    # 2) Agrupamos por 'jobrole'
    # 3) Calculamos la media de (s == "yes") que equivale al % de 'yes'
    # 4) Multiplicamos por 100 para tener porcentaje y ordenamos de mayor a menor
    attr_by_role = (
        df.assign(attr=df["attrition"].astype(str))
          .groupby("jobrole")["attr"]
          .apply(lambda s: (s == "yes").mean() * 100)
          .sort_values(ascending=False)
    )

    # Dibujamos el gráfico de barras con los porcentajes
    attr_by_role.plot(kind="bar")
    plt.title("Attrition por JobRole (%)")
    plt.xlabel("jobrole")         # eje X: nombre del rol
    plt.ylabel("% yes")           # eje Y: porcentaje de bajas (yes)
    plt.xticks(rotation=45, ha="right")  # rotamos etiquetas del X para que no se monten
    plt.tight_layout()                   # ajustamos diseño
    # Guardamos la imagen
    p4 = os.path.join(carpeta_plots, "attrition_by_jobrole_pct_v6.png")
    plt.savefig(p4, bbox_inches="tight")
    plt.close()
    print("Gráfico:", p4)

# --- Visualización 5: TotalWorkingYears vs YearsAtCompany (dispersión) ---
# Relación entre años totales de experiencia y años en la empresa actual.
if "totalworkingyears" in df.columns and "yearsatcompany" in df.columns:
    plt.figure()  # nueva figura
    # Guardamos los ejes X e Y como arrays numéricos (float) para la dispersión
    x = df["totalworkingyears"].astype(float)
    y = df["yearsatcompany"].astype(float)
    # Dibujamos puntos (cada punto es una persona)
    plt.scatter(x, y, s=10, alpha=0.6)
    plt.title("TotalWorkingYears vs YearsAtCompany (scatter)")
    plt.xlabel("totalworkingyears")     # eje X: años totales de experiencia
    plt.ylabel("yearsatcompany")        # eje Y: años en la empresa actual
    # Guardamos la imagen
    p5 = os.path.join(carpeta_plots, "scatter_totalworkingyears_vs_yearsatcompany_v6.png")
    plt.savefig(p5, bbox_inches="tight")
    plt.close()
    print("Gráfico:", p5)

Gráfico: outputs/plots/attrition_counts_v6.png
Gráfico: outputs/plots/monthlyincome_hist_v6.png
Gráfico: outputs/plots/yearsatcompany_by_department_v6.png
Gráfico: outputs/plots/attrition_by_jobrole_pct_v6.png


/var/folders/18/2jngcg0x5csd3tyb3xj47rrm0000gn/T/ipykernel_1770/967865439.py:56: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot(data, labels=labels, vert=True)  # vert=True para cajas verticales


Gráfico: outputs/plots/scatter_totalworkingyears_vs_yearsatcompany_v6.png


In [254]:
# =========================
# 16) VISUALIZACIONES II 
# =========================

# 4. Edad con salario y género - scatterplot con hue para género

plt.figure(figsize=(12,6))
# Scatter plot de salario contra edad, con separación por género y transparencia para mejor visualización
sns.scatterplot(data=df, x='age', y='salary', hue='gender', alpha=0.7, palette={'male':'#1f77b4', 'female':'#ff7f0e'})

plt.title('Salario según edad y género')
plt.xlabel('Edad (años)')
plt.ylabel('Salario (USD)')

# Ruta de guardado de la figura
p4 = os.path.join(carpeta_plots, 'salary_age_gender.png')
plt.savefig(p4, bbox_inches='tight')
plt.close()

# 5. Satisfacción vs % aumento salarial
plt.figure(figsize=(10,6))
# Regplot con puntos semi transparentes y línea de regresión
sns.regplot(data=df, x='jobsatisfaction', y='percentsalaryhike', scatter_kws={'alpha': 0.6}, line_kws={'color': 'darkgreen'})

plt.title('Relación entre satisfacción y porcentaje de aumento salarial')
plt.xlabel('Satisfacción en el trabajo')
plt.ylabel('Porcentaje de aumento salarial')

# Ruta de guardado de la figura
p5 = os.path.join(carpeta_plots, 'satisfaction_vs_salaryhike_colored.png')
plt.savefig(p5, bbox_inches='tight')
plt.close()

# 6. Tipo de jornada vs satisfacción, salario y aumento salarial
fg4 = sns.FacetGrid(df, col='jobsatisfaction', height=4, col_wrap=3, palette='Set2')
fg4.map_dataframe(sns.scatterplot, x='standardhours', y='salary', alpha=0.6, color='darkblue')
fg4.set_axis_labels('Tipo de jornada', 'Salario')
fg4.fig.suptitle('Salario vs tipo de jornada por nivel de satisfacción', fontsize=16)

# Ruta de guardado de la figura
p6 = os.path.join(carpeta_plots, 'standardhours_salary_satisfaction_colored.png')
fg4.savefig(p6)
plt.close()

# 7. Aumento salarial vs total de años trabajados
plt.figure(figsize=(10,6))
sns.regplot(data=df, x='totalworkingyears', y='percentsalaryhike', scatter_kws={'alpha': 0.6})
plt.title('Aumento salarial vs total de años trabajados')
plt.xlabel('Total de años trabajados')
plt.ylabel('Porcentaje de aumento de salario')
p7 = os.path.join(carpeta_plots, 'salaryhike_vs_workingyears.png')
plt.savefig(p7, bbox_inches='tight')
plt.close()

# 8. Total años trabajados por puesto de trabajo
avg_jobrole_yrs = df.groupby('jobrole')['totalworkingyears'].mean().reset_index()
plt.figure(figsize=(14,6))
sns.barplot(data=avg_jobrole_yrs, x='jobrole', y='totalworkingyears')
plt.title('Promedio de años trabajados vs puesto de trabajo')
plt.xticks(rotation=45)
plt.xlabel('Puesto de trabajo')
plt.ylabel('Promedio de años trabajados')
p9 = os.path.join(carpeta_plots, 'avg_years_by_jobrole.png')
plt.savefig(p9, bbox_inches='tight')
plt.close()

# 9. Jobrole con satisfacción
plt.figure(figsize=(14,6))
sns.stripplot(data=df, x='jobrole', y='jobsatisfaction', jitter=True, alpha=0.7)
plt.title('Relación entre puesto de trabajo y satisfacción')
plt.xticks(rotation=45)
plt.xlabel('Puesto de trabajo')
plt.ylabel('Satisfacción en el trabajo')
p10 = os.path.join(carpeta_plots, 'jobrole_vs_jobsatisfaction.png')
plt.savefig(p10, bbox_inches='tight')
plt.close()

# 10. Compromiso con el trabajo por puesto de trabajo, años trabajados y salario
plt.figure(figsize=(14,6))
sns.stripplot(data=df, x='jobrole', y='jobinvolvement', jitter=True, alpha=0.7)
plt.title('Compromiso con el trabajo vs puesto de trabajo')
plt.xticks(rotation=45)
plt.xlabel('Puesto de trabajo')
plt.ylabel('Compromiso con el trabajo')
p11 = os.path.join(carpeta_plots, 'jobinvolvement_vs_jobrole.png')
plt.savefig(p11, bbox_inches='tight')
plt.close()

plt.figure(figsize=(10,6))
sns.scatterplot(data=df, x='totalworkingyears', y='jobinvolvement', alpha=0.6)
plt.title('Compromiso con el trabajo vs total de años trabajados')
plt.xlabel('Total de años trabajados')
plt.ylabel('Compromiso con el trabajo')
p12 = os.path.join(carpeta_plots, 'jobinvolvement_vs_workingyears.png')
plt.savefig(p12, bbox_inches='tight')
plt.close()

plt.figure(figsize=(10,6))
sns.scatterplot(data=df, x='salary', y='jobinvolvement', alpha=0.6)
plt.title('Compromiso con el trabajo vs salario')
plt.xlabel('Salario')
plt.ylabel('Compromiso con el trabajo')
p13 = os.path.join(carpeta_plots, 'jobinvolvement_vs_salary.png')
plt.savefig(p13, bbox_inches='tight')
plt.close()

# 11. Salario con evaluación de desempeño
sns.scatterplot(data=df, x='performancerating', y='salary', alpha=0.6)
plt.title('Salario vs evaluación de desempeño')
plt.xlabel('Evaluación de desempeño')
plt.ylabel('Salario')
p14 = os.path.join(carpeta_plots, 'salary_vs_performance.png')
plt.savefig(p14, bbox_inches='tight')
plt.close()

# 12. Evaluación de desempeño con puesto de trabajo y total de años trabajados
plt.figure(figsize=(14,6))
sns.stripplot(data=df, x='jobrole', y='performancerating', jitter=True, alpha=0.7)
plt.title('Evaluación de desempeño vs puesto de trabajo')
plt.xticks(rotation=45)
plt.xlabel('Puesto de trabajo')
plt.ylabel('Evaluación de desempeño')
p15 = os.path.join(carpeta_plots, 'performance_vs_jobrole.png')
plt.savefig(p15, bbox_inches='tight')
plt.close()

plt.figure(figsize=(10,6))
sns.scatterplot(data=df, x='totalworkingyears', y='performancerating', alpha=0.6)
plt.title('Evaluación de desempeño vs total de años trabajados')
plt.xlabel('Total de años trabajados')
plt.ylabel('Evaluación de desempeño')
p16 = os.path.join(carpeta_plots, 'performance_vs_workingyears.png')
plt.savefig(p16, bbox_inches='tight')
plt.close()

# 13. Años desde último ascenso con satisfacción, aumento y jobrole

fg11 = sns.FacetGrid(df, col='jobrole', height=4, col_wrap=3)
fg11.map_dataframe(sns.scatterplot, x='yearssincelastpromotion', y='jobsatisfaction', alpha=0.6)
fg11.set_axis_labels('Años desde último ascenso', 'Satisfacción en el trabajo')
plt.subplots_adjust(top=0.9)
fg11.fig.suptitle('Años desde último ascenso vs satisfacción por puesto de trabajo')
p17 = os.path.join(carpeta_plots, 'years_since_promotion_vs_satisfaction.png')
fg11.savefig(p17)
plt.close()

plt.figure(figsize=(14,6))
sns.scatterplot(data=df, x='yearssincelastpromotion', y='percentsalaryhike', alpha=0.6)
plt.title('Años desde último ascenso vs % aumento salarial')
plt.xlabel('Años desde último ascenso')
plt.ylabel('Porcentaje de aumento salarial')
p18 = os.path.join(carpeta_plots, 'years_since_promotion_vs_salaryhike.png')
plt.savefig(p18, bbox_inches='tight')
plt.close()

plt.figure(figsize=(14,6))
sns.stripplot(data=df, x='jobrole', y='yearssincelastpromotion', jitter=True, alpha=0.7)
plt.title('Distribución de años desde último ascenso vs puesto de trabajo')
plt.xticks(rotation=45)
plt.xlabel('Puesto de trabajo')
plt.ylabel('Años desde último ascenso')
p19 = os.path.join(carpeta_plots, 'years_since_promotion_by_jobrole.png')
plt.savefig(p19, bbox_inches='tight')
plt.close()

# 14. Género con salario y nivel de estudios (3 variables)
plt.figure(figsize=(12,6))
sns.scatterplot(data=df, x='education', y='salary', hue='gender', alpha=0.7)
plt.title('Salario vs nivel de estudios con género')
plt.xlabel('Nivel de estudios')
plt.ylabel('Salario')
p20 = os.path.join(carpeta_plots, 'salary_education_gender.png')
plt.savefig(p20, bbox_inches='tight')
plt.close()

# 15. Distribución de años desde el último ascenso
plt.figure(figsize=(12,6))
sns.histplot(df['yearssincelastpromotion'], bins=15, kde=False)
plt.title('Distribución de años desde último ascenso')
plt.xlabel('Años desde último ascenso')
plt.ylabel('Cantidad de empleados')
plt.xlim(left=0)  # Esto hace que el eje X empiece en 0 en la esquina izquierda
p21 = os.path.join(carpeta_plots, 'distribution_years_since_last_promotion.png')
plt.savefig(p21, bbox_inches='tight')
plt.close()

# 16. Relación entre categoría laboral y salario (remuneración diaria)
plt.figure(figsize=(12,6))
sns.scatterplot(data=df, x='joblevel', y='dailyrate', alpha=0.7)
plt.title('Relación entre categoría laboral y remuneración diaria')
plt.xlabel('Categoría laboral')
plt.ylabel('Remuneración diaria')
p23 = os.path.join(carpeta_plots, 'joblevel_vs_dailyrate.png')
plt.savefig(p23, bbox_inches='tight')
plt.close()

# 17. Deserción por categoría laboral y salario
plt.figure(figsize=(12,6))
sns.stripplot(data=df, x='joblevel', y='salary', hue='attrition', size=4, dodge=True, jitter=True)
plt.title('Salario y categoría laboral vs. deserción')
plt.xlabel('Categoría laboral')
plt.ylabel('Salario')
p25 = os.path.join(carpeta_plots, 'attrition_with_joblevel_salary.png')
plt.savefig(p25, bbox_inches='tight')
plt.close()

# 18. Satisfacción vs performance rating con remote work como hue
plt.figure(figsize=(12,6))
sns.boxplot(data=df, x='performancerating', y='jobsatisfaction', hue='remotework')
plt.title('Satisfacción vs evaluación de desempeño según trabajo a distancia')
plt.xlabel('Evaluación de desempeño')
plt.ylabel('Satisfacción en el Trabajo')
p27 = os.path.join(carpeta_plots, 'jobsatisfaction_vs_performance_remote.png')
plt.savefig(p27, bbox_inches='tight')
plt.close()

# 19. Relación entre deserción y puestos de trabajo
# Calcular conteo total y conteo de desertores por job role
jobrole_total = df.groupby('jobrole')['attrition'].count()
jobrole_attrition = df[df['attrition'] == 'yes'].groupby('jobrole')['attrition'].count()

# Calcular proporción (tasa) de deserción
attrition_rate = (jobrole_attrition / jobrole_total).fillna(0)

# Pasar a DataFrame para graficar
attrition_rate_df = attrition_rate.reset_index()
attrition_rate_df.columns = ['jobrole', 'attrition_rate']

plt.figure(figsize=(14,7))
sns.barplot(data=attrition_rate_df, x='jobrole', y='attrition_rate')
plt.title('Tasa Relativa de deserción por puesto de trabajo')
plt.xlabel('Puesto de trabajo')
plt.ylabel('Tasa de deserción (proporción)')
plt.xticks(rotation=45, ha='right')

p29 = os.path.join(carpeta_plots, 'relative_attrition_by_jobrole.png')
plt.savefig(p29, bbox_inches='tight')
plt.close()

# 20. Estado civil cruzado con salario
plt.figure(figsize=(12,6))
sns.boxplot(data=df, x='maritalstatus', y='salary')
plt.title('Salario vs estado civil')
plt.xlabel('Estado civil')
plt.ylabel('Salario')
p30 = os.path.join(carpeta_plots, 'salary_vs_maritalstatus.png')
plt.savefig(p30, bbox_inches='tight')
plt.close()

# 21. Horas extra cruzado con deserción
plt.figure(figsize=(12,6))
sns.countplot(data=df, x='overtime', hue='attrition')
plt.title('Deserción según horas extra')
plt.xlabel('Horas extra')
plt.ylabel('Cantidad de empleados')
p31 = os.path.join(carpeta_plots, 'attrition_vs_overtime.png')
plt.savefig(p31, bbox_inches='tight')
plt.close()

# 22. Relación entre satisfacción en el trabajo y distancia a casa
plt.figure(figsize=(12,6))
sns.boxplot(data=df, x='distancefromhome', y='jobsatisfaction')
plt.title('Satisfacción laboral vs distancia al trabajo')
plt.xlabel('Distancia desde casa')
plt.ylabel('Satisfacción en el trabajo')
plt.xticks(rotation=45)
plt.tight_layout()
p32 = os.path.join(carpeta_plots, 'satisfaction_vs_distancefromhome.png')
plt.savefig(p32, bbox_inches='tight')
plt.close()

# 23. Relación entre capacitación en el último año y evaluación del empleado
plt.figure(figsize=(12,6))
sns.countplot(data=df, x='trainingtimeslastyear', hue='performancerating')
plt.title('Capacitación el último año vs evaluación del empleado')
plt.xlabel('Veces capacitado el último año')
plt.ylabel('Cantidad de empleados')
plt.legend(title='Evaluación')
p33 = os.path.join(carpeta_plots, 'training_vs_performance.png')
plt.savefig(p33, bbox_inches='tight')
plt.close()

In [255]:
# =========================
# 17) VERIFICACIÓN LIMPIEZA (DEMO) — antes vs después
# =========================
# OBJETIVO DE ESTA CELDA:
# - Preparar material para la DEMO del proyecto.
# - Comparar algunos campos importantes ENTRE el dataset bruto (df_raw) y el limpio (df).
# - Mostrar pequeñas tablas con ejemplos "antes vs después" para enseñar la calidad de la limpieza.
# - Guardar esas tablas en CSV dentro de 'outputs/verification' para poder adjuntarlas al repo/presentación.
# - Además, revisar rápidamente que las columnas tipo Likert (1..4) hayan quedado con valores válidos.

from IPython.display import display  # 'display' muestra dataframes bonitos en el notebook

def muestras_cambios(col, n=8):
    """
    Esta función devuelve una tabla con EJEMPLOS donde el valor CAMBIÓ tras la limpieza.
    - col: nombre de la columna a comparar entre df_raw (antes) y df (después).
    - n: cuántas filas de ejemplo queremos ver (por defecto 8).
    """
    # 1) Comprobamos que la columna exista en AMBOS dataframes
    if col not in df.columns or col not in df_raw.columns:
        print(f" - Columna '{col}' no existe en df o df_raw.")
        return None

    # 2) Convertimos a string para comparar sin errores de tipo
    orig = df_raw[col].astype("string")  # valores "antes" (bruto)
    new  = df[col].astype("string")      # valores "después" (limpio)

    # 3) Unimos en una tabla con dos columnas: original vs limpio
    cambios = pd.DataFrame({"original": orig, "limpio": new})

    # 4) Nos quedamos solo con las filas donde los valores SON DISTINTOS
    #    (dropna(how="all") evita mostrar filas que sean todo NaN)
    cambios = cambios[cambios["original"] != cambios["limpio"]].dropna(how="all")

    # 5) Si no hay cambios, lo decimos por pantalla y devolvemos None
    if cambios.empty:
        print(f" - Sin cambios detectados en '{col}' (o ya estaba limpio).")
        return None

    # 6) Devolvemos SOLO n ejemplos (para no saturar la pantalla)
    return cambios.head(n)

# -------------------------------------------------------------
# 1) Elegimos QUÉ columnas queremos comparar en la demo
# -------------------------------------------------------------
# Son columnas representativas donde normalmente hay normalización/limpieza:
# - roledepartament: suele traer guiones/espacios/mezcla de mayúsculas.
# - maritalstatus: minúsculas + corrección del typo "marreid" -> "married".
# - businesstravel: normalización (p. ej., "non-travel" -> "non_travel").
# - educationfield y standardhours: sustitución de espacios por "_", minúsculas, etc.
columns_demo = ["roledepartament", "maritalstatus", "businesstravel", "educationfield", "standardhours"]

# Diccionario para guardar las mini-tablas que generemos (clave = nombre columna)
tablas_demo = {}

# Recorremos cada columna seleccionada y mostramos/guardamos los cambios
for c in columns_demo:
    print(f"\n### {c} — antes vs después")     # título en consola para separar bloques
    df_demo = muestras_cambios(c, n=8)         # pedimos 8 ejemplos
    if df_demo is not None:
        display(df_demo)                       # mostramos la tabla en el notebook
        tablas_demo[c] = df_demo               # guardamos la tabla en el diccionario

# -------------------------------------------------------------
# 2) Guardamos las mini-tablas en CSV para la presentación
# -------------------------------------------------------------
# Creamos (si no existe) la carpeta 'outputs/verification'
out_dir = os.path.join(carpeta_outputs, "verification")
os.makedirs(out_dir, exist_ok=True)

# Recorremos las tablas generadas y las guardamos una por una
for c, t in tablas_demo.items():
    p = os.path.join(out_dir, f"muestras_{c}_v6.csv")
    t.to_csv(p, index=False)                   # index=False para no añadir la columna del índice
    print("Guardado:", p)                      # confirmamos la ruta

# -------------------------------------------------------------
# 3) Chequeo especial: columnas LIKERT (valores válidos 1..4)
# -------------------------------------------------------------
# Aquí revisamos que, en el dataset BRUTO, existieran valores raros (fuera de 1..4),
# y que en el dataset LIMPIO, los valores hayan quedado dentro de lo esperado.
likert_cols = ["environmentsatisfaction", "jobsatisfaction", "relationshipsatisfaction", "jobinvolvement", "worklifebalance"]

for col in likert_cols:
    # Solo procedemos si la columna existe en ambos dataframes
    if col in df_raw.columns and col in df.columns:

        # --- En el BRUTO: buscamos valores fuera de 1..4
        bruto_num = pd.to_numeric(df_raw[col], errors="coerce")  # convertimos a número (lo no convertible pasa a NaN)
        fuera_rango = bruto_num[~bruto_num.isna() & ~bruto_num.isin([1, 2, 3, 4])]
        if len(fuera_rango) > 0:
            print(f"\n[LIKERT] '{col}' en bruto: {len(fuera_rango)} valores fuera de 1..4 (mostramos hasta 5)")
            display(fuera_rango.head(5))  # mostramos algunas muestras problemáticas
        else:
            print(f"\n[LIKERT] '{col}' en bruto: sin valores raros detectados")

        # --- En el LIMPIO: vemos qué valores únicos quedaron (deberían ser 1..4)
        limpio_num = pd.to_numeric(df[col], errors="coerce")
        unicos = sorted(pd.Series(limpio_num.unique()).dropna().tolist())
        print(f"[LIKERT] '{col}' en limpio: valores únicos = {unicos}")

# NOTA:
# - Este bloque es MUY útil para enseñar a profesores/compañeras la diferencia real
#   entre el “antes” y el “después”.
# - Si queréis, podéis añadir más columnas a 'columns_demo' según lo que queráis destacar.


### roledepartament — antes vs después
 - Columna 'roledepartament' no existe en df o df_raw.

### maritalstatus — antes vs después


,original,limpio
2,Married,married
3,Married,married
4,Divorced,divorced
7,Married,married
8,Married,married
11,Married,married
12,Married,married
13,Married,married



### businesstravel — antes vs después


,original,limpio
10,non-travel,non_travel
20,non-travel,non_travel
29,non-travel,non_travel
54,non-travel,non_travel
55,non-travel,non_travel
70,non-travel,non_travel
139,non-travel,non_travel
146,non-travel,non_travel



### educationfield — antes vs después


...


### standardhours — antes vs después


,original,limpio
0,Full Time,full_time
3,Full Time,full_time
6,Part Time,part_time
7,Part Time,part_time
8,Full Time,full_time
10,Full Time,full_time
11,Full Time,full_time
12,Full Time,full_time


Guardado: outputs/verification/muestras_maritalstatus_v6.csv
Guardado: outputs/verification/muestras_businesstravel_v6.csv
Guardado: outputs/verification/muestras_educationfield_v6.csv
Guardado: outputs/verification/muestras_standardhours_v6.csv

[LIKERT] 'environmentsatisfaction' en bruto: 102 valores fuera de 1..4 (mostramos hasta 5)


24    42
30    37
41    35
68    25
90    27
Name: environmentsatisfaction, dtype: int64

[LIKERT] 'environmentsatisfaction' en limpio: valores únicos = [1.0, 2.0, 3.0, 4.0]

[LIKERT] 'jobsatisfaction' en bruto: sin valores raros detectados
[LIKERT] 'jobsatisfaction' en limpio: valores únicos = [1.0, 2.0, 3.0, 4.0]

[LIKERT] 'relationshipsatisfaction' en bruto: sin valores raros detectados
[LIKERT] 'relationshipsatisfaction' en limpio: valores únicos = [1.0, 2.0, 3.0, 4.0]

[LIKERT] 'jobinvolvement' en bruto: sin valores raros detectados
[LIKERT] 'jobinvolvement' en limpio: valores únicos = [1.0, 2.0, 3.0, 4.0]

[LIKERT] 'worklifebalance' en bruto: sin valores raros detectados
[LIKERT] 'worklifebalance' en limpio: valores únicos = [1.0, 2.0, 3.0, 4.0]
